In [1]:
import numpy as np
import pandas as pd
from scipy import sparse
import xgboost as xgb
import random
import math
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import model_selection, preprocessing, ensemble

/Users/appe/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
train_df = pd.read_json("data/train.json")
test_df = pd.read_json("data/test.json")
image_date = pd.read_csv("data/listing_image_time.csv")

In [4]:
train_df.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,interest_level,latitude,listing_id,longitude,manager_id,photos,price,street_address
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],medium,40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000,792 Metropolitan Avenue
10000,1.0,2,c5c8a357cba207596b04d1afd1e4f130,2016-06-12 12:19:27,,Columbus Avenue,"[Doorman, Elevator, Fitness Center, Cats Allow...",low,40.7947,7150865,-73.9667,7533621a882f71e25173b27e3139d83d,[https://photos.renthop.com/2/7150865_be3306c5...,5465,808 Columbus Avenue
100004,1.0,1,c3ba40552e2120b0acfc3cb5730bb2aa,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,"[Laundry In Building, Dishwasher, Hardwood Flo...",high,40.7388,6887163,-74.0018,d9039c43983f6e564b1482b273bd7b01,[https://photos.renthop.com/2/6887163_de85c427...,2850,241 W 13 Street
100007,1.0,1,28d9ad350afeaab8027513a3e52ac8d5,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,"[Hardwood Floors, No Fee]",low,40.7539,6888711,-73.9677,1067e078446a7897d2da493d2f741316,[https://photos.renthop.com/2/6888711_6e660cee...,3275,333 East 49th Street
100013,1.0,4,0,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,[Pre-War],low,40.8241,6934781,-73.9493,98e13ad4b495b9613cef886d79a6291f,[https://photos.renthop.com/2/6934781_1fa4b41a...,3350,500 West 143rd Street


In [5]:
test_df.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address
0,1.0,1,79780be1514f645d7e6be99a3de696c5,2016-06-11 05:29:41,Large with awesome terrace--accessible via bed...,Suffolk Street,"[Elevator, Laundry in Building, Laundry in Uni...",40.7185,7142618,-73.9865,b1b1852c416d78d7765d746cb1b8921f,[https://photos.renthop.com/2/7142618_1c45a2c8...,2950,99 Suffolk Street
1,1.0,2,0,2016-06-24 06:36:34,Prime Soho - between Bleecker and Houston - Ne...,Thompson Street,"[Pre-War, Dogs Allowed, Cats Allowed]",40.7278,7210040,-74.0000,d0b5648017832b2427eeb9956d966a14,[https://photos.renthop.com/2/7210040_d824cc71...,2850,176 Thompson Street
100,1.0,1,3dbbb69fd52e0d25131aa1cd459c87eb,2016-06-03 04:29:40,New York chic has reached a new level ...,101 East 10th Street,"[Doorman, Elevator, No Fee]",40.7306,7103890,-73.9890,9ca6f3baa475c37a3b3521a394d65467,[https://photos.renthop.com/2/7103890_85b33077...,3758,101 East 10th Street
1000,1.0,2,783d21d013a7e655bddc4ed0d461cc5e,2016-06-11 06:17:35,Step into this fantastic new Construction in t...,South Third Street\r,"[Roof Deck, Balcony, Elevator, Laundry in Buil...",40.7109,7143442,-73.9571,0b9d5db96db8472d7aeb67c67338c4d2,[https://photos.renthop.com/2/7143442_0879e9e0...,3300,251 South Third Street\r
100000,2.0,2,6134e7c4dd1a98d9aee36623c9872b49,2016-04-12 05:24:17,"~Take a stroll in Central Park, enjoy the ente...","Midtown West, 8th Ave","[Common Outdoor Space, Cats Allowed, Dogs Allo...",40.7650,6860601,-73.9845,b5eda0eb31b042ce2124fd9e9fcfce2f,[https://photos.renthop.com/2/6860601_c96164d8...,4900,260 West 54th Street


In [7]:
# rename columns so you can join tables later on
image_date.columns = ["listing_id", "time_stamp"]

# reassign the only one timestamp from April, all others from Oct/Nov
image_date.loc[80240,"time_stamp"] = 1478129766 

image_date["img_date"]                  = pd.to_datetime(image_date["time_stamp"], unit="s")
image_date["img_days_passed"]           = (image_date["img_date"].max() - image_date["img_date"]).astype("timedelta64[D]").astype(int)
image_date["img_date_month"]            = image_date["img_date"].dt.month
image_date["img_date_week"]             = image_date["img_date"].dt.week
image_date["img_date_day"]              = image_date["img_date"].dt.day
image_date["img_date_dayofweek"]        = image_date["img_date"].dt.dayofweek
image_date["img_date_dayofyear"]        = image_date["img_date"].dt.dayofyear
image_date["img_date_hour"]             = image_date["img_date"].dt.hour
image_date["img_date_monthBeginMidEnd"] = image_date["img_date_day"].apply(lambda x: 1 if x<10 else 2 if x<20 else 3)



In [10]:
train_df = train_df.join(image_date.set_index('listing_id'), on='listing_id')
test_df = test_df.join(image_date.set_index('listing_id'), on='listing_id')

In [13]:
def cart2rho(x, y):
    rho = np.sqrt(x**2 + y**2)
    return rho


def cart2phi(x, y):
    phi = np.arctan2(y, x)
    return phi


def rotation_x(row, alpha):
    x = row['latitude']
    y = row['longitude']
    return x*math.cos(alpha) + y*math.sin(alpha)


def rotation_y(row, alpha):
    x = row['latitude']
    y = row['longitude']
    return y*math.cos(alpha) - x*math.sin(alpha)


def add_rotation(degrees, df):
    namex = "rot" + str(degrees) + "_X"
    namey = "rot" + str(degrees) + "_Y"

    df['num_' + namex] = df.apply(lambda row: rotation_x(row, math.pi/(180/degrees)), axis=1)
    df['num_' + namey] = df.apply(lambda row: rotation_y(row, math.pi/(180/degrees)), axis=1)

    return df

def operate_on_coordinates(tr_df, te_df):
    for df in [tr_df, te_df]:
        #polar coordinates system
        df["num_rho"] = df.apply(lambda x: cart2rho(x["latitude"] - 40.78222222, x["longitude"]+73.96527777), axis=1)
        df["num_phi"] = df.apply(lambda x: cart2phi(x["latitude"] - 40.78222222, x["longitude"]+73.96527777), axis=1)
        #rotations
        for angle in [15,30,45,60]:
            df = add_rotation(angle, df)

    return tr_df, te_df

In [14]:
train_df, test_df = operate_on_coordinates(train_df, test_df)

In [34]:
def cap_share(x):
    return sum(1 for c in x if c.isupper())/float(len(x)+1)

def try_and_find_nr(description):
    if reg.match(description) is None:
        return 0
    return 1

In [35]:
for df in [train_df, test_df]:
    # do you think that users might feel annoyed BY A DESCRIPTION THAT IS SHOUTING AT THEM?
    df['num_cap_share'] = df['description'].apply(cap_share)
    
    # how long in lines the desc is?
    df['num_nr_of_lines'] = df['description'].apply(lambda x: x.count('<br /><br />'))
   
    # is the description redacted by the website?        
    df['num_redacted'] = 0
    df['num_redacted'].ix[df['description'].str.contains('website_redacted')] = 1

    
    # can we contact someone via e-mail to ask for the details?
    df['num_email'] = 0
    df['num_email'].ix[df['description'].str.contains('@')] = 1
    
    #and... can we call them?
    
    reg = re.compile(".*?(\(?\d{3}\D{0,3}\d{3}\D{0,3}\d{4}).*?", re.S)


    df['num_phone_nr'] = df['description'].apply(try_and_find_nr)

/Users/appe/anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [39]:
features_to_use = list(train_df.columns)[-1:-16:-1]

In [40]:
train_df.loc[104459, 'bathrooms'] = 2
test_df.loc[17808, 'bathrooms'] = 3.0
test_df.loc[22737, 'bathrooms'] = 2.0
test_df.loc[837, 'bathrooms'] = 3.0

In [41]:
train_df.loc[train_df.bathrooms < 1, 'bathrooms'] = 0.5
test_df.loc[test_df.bathrooms < 1, 'bathrooms'] = 0.5
train_df.loc[train_df.bedrooms < 1, 'bedrooms'] = 0.5
test_df.loc[test_df.bedrooms < 1, 'bedrooms'] = 0.5

In [42]:
train_df.loc[train_df.price > 100000, 'price'] = np.floor(train_df.loc[train_df.price > 100000, 'price'] / 100)
train_df["price"] = train_df["price"].clip(upper=31000)
test_df.loc[test_df.price > 100000, 'price'] = np.floor(test_df.loc[test_df.price > 100000, 'price'] / 100)
test_df["price"] = test_df["price"].clip(upper=31000)

In [43]:
test_df.loc[44841, 'price'] = 1000
train_df.loc[87511, 'price'] = 450
train_df.loc[53144, 'price'] = 430

In [44]:
train_df["bathroom_per_bedroom"] = train_df["bathrooms"]/train_df["bedrooms"]
test_df["bathroom_per_bedroom"] = test_df["bathrooms"]/test_df["bedrooms"] 

In [45]:
train_df["logprice"] = np.log(train_df["price"])
test_df["logprice"] = np.log(test_df["price"])

train_df["price_t"] =train_df["price"]/train_df["bedrooms"]
test_df["price_t"] = test_df["price"]/test_df["bedrooms"] 

train_df["room_sum"] = train_df["bedrooms"]+train_df["bathrooms"] 
test_df["room_sum"] = test_df["bedrooms"]+test_df["bathrooms"] 

train_df['price_per_room'] = train_df['price']/train_df['room_sum']
test_df['price_per_room'] = test_df['price']/test_df['room_sum']

train_df["num_photos"] = train_df["photos"].apply(len)
test_df["num_photos"] = test_df["photos"].apply(len)

train_df["num_features"] = train_df["features"].apply(len)
test_df["num_features"] = test_df["features"].apply(len)

train_df["num_description_words"] = train_df["description"].apply(lambda x: len(x.split(" ")))
test_df["num_description_words"] = test_df["description"].apply(lambda x: len(x.split(" ")))

train_df["created"] = pd.to_datetime(train_df["created"])
test_df["created"] = pd.to_datetime(test_df["created"])
train_df["created_year"] = train_df["created"].dt.year
test_df["created_year"] = test_df["created"].dt.year
train_df["created_month"] = train_df["created"].dt.month
test_df["created_month"] = test_df["created"].dt.month
train_df["created_day"] = train_df["created"].dt.day
test_df["created_day"] = test_df["created"].dt.day
train_df["created_hour"] = train_df["created"].dt.hour
test_df["created_hour"] = test_df["created"].dt.hour

In [48]:
train_df["pos"] = train_df.longitude.round(3).astype(str) + '_' + train_df.latitude.round(3).astype(str)
test_df["pos"] = test_df.longitude.round(3).astype(str) + '_' + test_df.latitude.round(3).astype(str)

vals = train_df['pos'].value_counts()
dvals = vals.to_dict()

In [54]:
train_df["density"] = train_df['pos'].apply(lambda x: dvals.get(x, vals.min()))
test_df["density"] = test_df['pos'].apply(lambda x: dvals.get(x, vals.min()))

In [55]:
features_to_use.extend(["bathrooms", "bedrooms", "latitude", "longitude", "price","price_t", "price_per_room", "logprice", "density",
"num_photos", "num_features", "num_description_words","listing_id", "created_month", "created_day", "created_hour", 'time_stamp'])

features_to_use.append('bathroom_per_bedroom')
features_to_use.extend(['img_days_passed', 'img_date_month', 'img_date_week', 'img_date_day', 'img_date_hour',
                        'img_date_dayofweek', 'img_date_dayofyear', 'img_date_monthBeginMidEnd'])



In [58]:
index=list(range(train_df.shape[0]))
random.shuffle(index)
a=[np.nan]*len(train_df)
b=[np.nan]*len(train_df)
c=[np.nan]*len(train_df)

In [70]:
for i in range(5):
    building_level={}
    for j in train_df['manager_id'].values:
        building_level[j]=[0,0,0]
    
    test_index=index[int((i*train_df.shape[0])/5):int(((i+1)*train_df.shape[0])/5)]
    train_index=list(set(index).difference(test_index))
    
    for j in train_index:
        temp=train_df.iloc[j]
        if temp['interest_level']=='low':
            building_level[temp['manager_id']][0]+=1
        if temp['interest_level']=='medium':
            building_level[temp['manager_id']][1]+=1
        if temp['interest_level']=='high':
            building_level[temp['manager_id']][2]+=1
            
    for j in test_index:
        temp=train_df.iloc[j]
        if sum(building_level[temp['manager_id']])!=0:
            a[j]=building_level[temp['manager_id']][0]*1.0/sum(building_level[temp['manager_id']])
            b[j]=building_level[temp['manager_id']][1]*1.0/sum(building_level[temp['manager_id']])
            c[j]=building_level[temp['manager_id']][2]*1.0/sum(building_level[temp['manager_id']])
            
train_df['manager_level_low']=a
train_df['manager_level_medium']=b
train_df['manager_level_high']=c

a=[]
b=[]
c=[]
building_level={}
for j in train_df['manager_id'].values:
    building_level[j]=[0,0,0]

for j in range(train_df.shape[0]):
    temp=train_df.iloc[j]
    if temp['interest_level']=='low':
        building_level[temp['manager_id']][0]+=1
    if temp['interest_level']=='medium':
        building_level[temp['manager_id']][1]+=1
    if temp['interest_level']=='high':
        building_level[temp['manager_id']][2]+=1

for i in test_df['manager_id'].values:
    if i not in building_level.keys():
        a.append(np.nan)
        b.append(np.nan)
        c.append(np.nan)
    else:
        a.append(building_level[i][0]*1.0/sum(building_level[i]))
        b.append(building_level[i][1]*1.0/sum(building_level[i]))
        c.append(building_level[i][2]*1.0/sum(building_level[i]))
test_df['manager_level_low']=a
test_df['manager_level_medium']=b
test_df['manager_level_high']=c

features_to_use.append('manager_level_low') 
features_to_use.append('manager_level_medium') 
features_to_use.append('manager_level_high')

In [71]:
categorical = ["display_address", "manager_id", "building_id"]
for f in categorical:
        if train_df[f].dtype=='object':
            lbl = LabelEncoder()
            lbl.fit(list(train_df[f].values) + list(test_df[f].values))
            train_df[f] = lbl.transform(list(train_df[f].values))
            test_df[f] = lbl.transform(list(test_df[f].values))
            features_to_use.append(f)

In [75]:
train_df['features'] = train_df["features"].apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))
test_df['features'] = test_df["features"].apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))

In [76]:
tfidf = CountVectorizer(stop_words='english', max_features=200)
tr_sparse = tfidf.fit_transform(train_df["features"])
te_sparse = tfidf.transform(test_df["features"])

train_X = sparse.hstack([train_df[features_to_use], tr_sparse]).tocsr()
test_X = sparse.hstack([test_df[features_to_use], te_sparse]).tocsr()

target_num_map = {'high':0, 'medium':1, 'low':2}
train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))

In [79]:
def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=321, num_rounds=2000):
    param = {}
    param['objective'] = 'multi:softprob'
    param['eta'] = 0.02
    param['max_depth'] = 6
    param['silent'] = 1
    param['num_class'] = 3
    param['eval_metric'] = "mlogloss"
    param['min_child_weight'] = 1
    param['subsample'] = 0.7
    param['colsample_bytree'] = 0.7
    param['seed'] = seed_val
    num_rounds = num_rounds

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=50)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

    pred_test_y = model.predict(xgtest)
    return pred_test_y, model

In [80]:
cv_scores = []
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2016)
for dev_index, val_index in kf.split(range(train_X.shape[0])):
        dev_X, val_X = train_X[dev_index,:], train_X[val_index,:]
        dev_y, val_y = train_y[dev_index], train_y[val_index]
        preds, model = runXGB(dev_X, dev_y, val_X, val_y)
        cv_scores.append(log_loss(val_y, preds))

[0]	train-mlogloss:1.08517	test-mlogloss:1.08536
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.
[1]	train-mlogloss:1.07204	test-mlogloss:1.07258
[2]	train-mlogloss:1.05912	test-mlogloss:1.0599
[3]	train-mlogloss:1.04647	test-mlogloss:1.04756
[4]	train-mlogloss:1.03497	test-mlogloss:1.03626
[5]	train-mlogloss:1.02349	test-mlogloss:1.02515
[6]	train-mlogloss:1.01181	test-mlogloss:1.0138
[7]	train-mlogloss:1.00082	test-mlogloss:1.0031
[8]	train-mlogloss:0.990119	test-mlogloss:0.992683
[9]	train-mlogloss:0.979442	test-mlogloss:0.982342
[10]	train-mlogloss:0.969033	test-mlogloss:0.972238
[11]	train-mlogloss:0.958987	test-mlogloss:0.962462
[12]	train-mlogloss:0.949668	test-mlogloss:0.953374
[13]	train-mlogloss:0.94102	test-mlogloss:0.944897
[14]	train-mlogloss:0.931668	test-mlogloss:0.935819
[15]	train-mlogloss:0.922734	test-mlogloss:0.92721
[16]	train-mlogloss:0.913999	test-mlogloss:0.918

[155]	train-mlogloss:0.547135	test-mlogloss:0.581958
[156]	train-mlogloss:0.54639	test-mlogloss:0.581401
[157]	train-mlogloss:0.545709	test-mlogloss:0.580867
[158]	train-mlogloss:0.545068	test-mlogloss:0.580376
[159]	train-mlogloss:0.544387	test-mlogloss:0.57982
[160]	train-mlogloss:0.543749	test-mlogloss:0.579395
[161]	train-mlogloss:0.543117	test-mlogloss:0.578908
[162]	train-mlogloss:0.542453	test-mlogloss:0.578453
[163]	train-mlogloss:0.541773	test-mlogloss:0.577939
[164]	train-mlogloss:0.541135	test-mlogloss:0.577466
[165]	train-mlogloss:0.540464	test-mlogloss:0.576987
[166]	train-mlogloss:0.539981	test-mlogloss:0.576608
[167]	train-mlogloss:0.539267	test-mlogloss:0.576111
[168]	train-mlogloss:0.538586	test-mlogloss:0.575619
[169]	train-mlogloss:0.538005	test-mlogloss:0.575224
[170]	train-mlogloss:0.537392	test-mlogloss:0.574806
[171]	train-mlogloss:0.536764	test-mlogloss:0.574359
[172]	train-mlogloss:0.53621	test-mlogloss:0.573955
[173]	train-mlogloss:0.535583	test-mlogloss:0.573

[311]	train-mlogloss:0.480839	test-mlogloss:0.542336
[312]	train-mlogloss:0.480449	test-mlogloss:0.542197
[313]	train-mlogloss:0.480109	test-mlogloss:0.542051
[314]	train-mlogloss:0.479851	test-mlogloss:0.541914
[315]	train-mlogloss:0.47957	test-mlogloss:0.541812
[316]	train-mlogloss:0.479167	test-mlogloss:0.541648
[317]	train-mlogloss:0.478836	test-mlogloss:0.541539
[318]	train-mlogloss:0.478526	test-mlogloss:0.541401
[319]	train-mlogloss:0.478228	test-mlogloss:0.541226
[320]	train-mlogloss:0.477924	test-mlogloss:0.541069
[321]	train-mlogloss:0.477675	test-mlogloss:0.540973
[322]	train-mlogloss:0.47738	test-mlogloss:0.540855
[323]	train-mlogloss:0.477132	test-mlogloss:0.540774
[324]	train-mlogloss:0.476832	test-mlogloss:0.540651
[325]	train-mlogloss:0.47655	test-mlogloss:0.540516
[326]	train-mlogloss:0.476324	test-mlogloss:0.540445
[327]	train-mlogloss:0.476005	test-mlogloss:0.540305
[328]	train-mlogloss:0.475779	test-mlogloss:0.540209
[329]	train-mlogloss:0.475524	test-mlogloss:0.540

[467]	train-mlogloss:0.443634	test-mlogloss:0.529646
[468]	train-mlogloss:0.443428	test-mlogloss:0.5296
[469]	train-mlogloss:0.443231	test-mlogloss:0.529544
[470]	train-mlogloss:0.443026	test-mlogloss:0.529498
[471]	train-mlogloss:0.442776	test-mlogloss:0.529426
[472]	train-mlogloss:0.442616	test-mlogloss:0.529394
[473]	train-mlogloss:0.442374	test-mlogloss:0.529354
[474]	train-mlogloss:0.442158	test-mlogloss:0.529318
[475]	train-mlogloss:0.441896	test-mlogloss:0.529251
[476]	train-mlogloss:0.441653	test-mlogloss:0.529237
[477]	train-mlogloss:0.441503	test-mlogloss:0.529187
[478]	train-mlogloss:0.441328	test-mlogloss:0.52916
[479]	train-mlogloss:0.4411	test-mlogloss:0.529129
[480]	train-mlogloss:0.440847	test-mlogloss:0.529071
[481]	train-mlogloss:0.440637	test-mlogloss:0.529028
[482]	train-mlogloss:0.440423	test-mlogloss:0.528952
[483]	train-mlogloss:0.440234	test-mlogloss:0.528895
[484]	train-mlogloss:0.440074	test-mlogloss:0.528838
[485]	train-mlogloss:0.43991	test-mlogloss:0.528776

[623]	train-mlogloss:0.416891	test-mlogloss:0.524103
[624]	train-mlogloss:0.416703	test-mlogloss:0.524047
[625]	train-mlogloss:0.416488	test-mlogloss:0.524024
[626]	train-mlogloss:0.416337	test-mlogloss:0.523997
[627]	train-mlogloss:0.416169	test-mlogloss:0.523967
[628]	train-mlogloss:0.415977	test-mlogloss:0.523938
[629]	train-mlogloss:0.415885	test-mlogloss:0.523916
[630]	train-mlogloss:0.415736	test-mlogloss:0.523889
[631]	train-mlogloss:0.415601	test-mlogloss:0.52386
[632]	train-mlogloss:0.415455	test-mlogloss:0.523833
[633]	train-mlogloss:0.415316	test-mlogloss:0.523828
[634]	train-mlogloss:0.41519	test-mlogloss:0.5238
[635]	train-mlogloss:0.414979	test-mlogloss:0.523781
[636]	train-mlogloss:0.41479	test-mlogloss:0.52374
[637]	train-mlogloss:0.414618	test-mlogloss:0.523751
[638]	train-mlogloss:0.414442	test-mlogloss:0.523736
[639]	train-mlogloss:0.414228	test-mlogloss:0.523719
[640]	train-mlogloss:0.41407	test-mlogloss:0.523678
[641]	train-mlogloss:0.413939	test-mlogloss:0.523669


...

[1040]	train-mlogloss:0.368215	test-mlogloss:0.506903
[1041]	train-mlogloss:0.368135	test-mlogloss:0.50688
[1042]	train-mlogloss:0.368014	test-mlogloss:0.50687
[1043]	train-mlogloss:0.367941	test-mlogloss:0.506855
[1044]	train-mlogloss:0.367824	test-mlogloss:0.506843
[1045]	train-mlogloss:0.367773	test-mlogloss:0.506824
[1046]	train-mlogloss:0.367656	test-mlogloss:0.506821
[1047]	train-mlogloss:0.367573	test-mlogloss:0.506826
[1048]	train-mlogloss:0.367498	test-mlogloss:0.506825
[1049]	train-mlogloss:0.367377	test-mlogloss:0.506813
[1050]	train-mlogloss:0.367269	test-mlogloss:0.5068
[1051]	train-mlogloss:0.367158	test-mlogloss:0.506773
[1052]	train-mlogloss:0.367069	test-mlogloss:0.506762
[1053]	train-mlogloss:0.366951	test-mlogloss:0.506761
[1054]	train-mlogloss:0.366832	test-mlogloss:0.506746
[1055]	train-mlogloss:0.366736	test-mlogloss:0.506728
[1056]	train-mlogloss:0.36661	test-mlogloss:0.506747
[1057]	train-mlogloss:0.366486	test-mlogloss:0.506727
[1058]	train-mlogloss:0.366347	te

[1193]	train-mlogloss:0.351916	test-mlogloss:0.505697
[1194]	train-mlogloss:0.351822	test-mlogloss:0.505691
[1195]	train-mlogloss:0.351729	test-mlogloss:0.505696
[1196]	train-mlogloss:0.351591	test-mlogloss:0.505671
[1197]	train-mlogloss:0.351461	test-mlogloss:0.505666
[1198]	train-mlogloss:0.3514	test-mlogloss:0.505651
[1199]	train-mlogloss:0.351282	test-mlogloss:0.50564
[1200]	train-mlogloss:0.351168	test-mlogloss:0.505597
[1201]	train-mlogloss:0.351039	test-mlogloss:0.505591
[1202]	train-mlogloss:0.35096	test-mlogloss:0.505583
[1203]	train-mlogloss:0.350867	test-mlogloss:0.505593
[1204]	train-mlogloss:0.350753	test-mlogloss:0.505596
[1205]	train-mlogloss:0.350666	test-mlogloss:0.505588
[1206]	train-mlogloss:0.350576	test-mlogloss:0.505585
[1207]	train-mlogloss:0.350455	test-mlogloss:0.505576
[1208]	train-mlogloss:0.350335	test-mlogloss:0.505551
[1209]	train-mlogloss:0.350266	test-mlogloss:0.505544
[1210]	train-mlogloss:0.350183	test-mlogloss:0.505544
[1211]	train-mlogloss:0.35006	te

[1346]	train-mlogloss:0.336803	test-mlogloss:0.504683
[1347]	train-mlogloss:0.33672	test-mlogloss:0.504673
[1348]	train-mlogloss:0.33661	test-mlogloss:0.504685
[1349]	train-mlogloss:0.336553	test-mlogloss:0.504706
[1350]	train-mlogloss:0.336491	test-mlogloss:0.504695
[1351]	train-mlogloss:0.336383	test-mlogloss:0.504694
[1352]	train-mlogloss:0.336292	test-mlogloss:0.504676
[1353]	train-mlogloss:0.336159	test-mlogloss:0.504663
[1354]	train-mlogloss:0.336053	test-mlogloss:0.504627
[1355]	train-mlogloss:0.335961	test-mlogloss:0.504634
[1356]	train-mlogloss:0.335915	test-mlogloss:0.504635
[1357]	train-mlogloss:0.335825	test-mlogloss:0.504643
[1358]	train-mlogloss:0.335772	test-mlogloss:0.504633
[1359]	train-mlogloss:0.335658	test-mlogloss:0.504643
[1360]	train-mlogloss:0.335541	test-mlogloss:0.504653
[1361]	train-mlogloss:0.335433	test-mlogloss:0.504684
[1362]	train-mlogloss:0.335383	test-mlogloss:0.504668
[1363]	train-mlogloss:0.335282	test-mlogloss:0.504665
[1364]	train-mlogloss:0.335199

[1499]	train-mlogloss:0.322635	test-mlogloss:0.504317
[1500]	train-mlogloss:0.32257	test-mlogloss:0.504331
[1501]	train-mlogloss:0.322492	test-mlogloss:0.504336
[1502]	train-mlogloss:0.322388	test-mlogloss:0.50434
[1503]	train-mlogloss:0.322248	test-mlogloss:0.50431
[1504]	train-mlogloss:0.32214	test-mlogloss:0.504307
[1505]	train-mlogloss:0.322045	test-mlogloss:0.504295
[1506]	train-mlogloss:0.321944	test-mlogloss:0.504304
[1507]	train-mlogloss:0.321848	test-mlogloss:0.504309
[1508]	train-mlogloss:0.321745	test-mlogloss:0.504312
[1509]	train-mlogloss:0.321659	test-mlogloss:0.504307
[1510]	train-mlogloss:0.321555	test-mlogloss:0.504291
[1511]	train-mlogloss:0.321505	test-mlogloss:0.504278
[1512]	train-mlogloss:0.321383	test-mlogloss:0.504286
[1513]	train-mlogloss:0.321286	test-mlogloss:0.504282
[1514]	train-mlogloss:0.321187	test-mlogloss:0.504274
[1515]	train-mlogloss:0.321081	test-mlogloss:0.504285
[1516]	train-mlogloss:0.321048	test-mlogloss:0.504284
[1517]	train-mlogloss:0.320957	t

[1652]	train-mlogloss:0.309337	test-mlogloss:0.503873
[1653]	train-mlogloss:0.309244	test-mlogloss:0.503877
[1654]	train-mlogloss:0.309204	test-mlogloss:0.503861
[1655]	train-mlogloss:0.309089	test-mlogloss:0.503867
[1656]	train-mlogloss:0.309016	test-mlogloss:0.503877
[1657]	train-mlogloss:0.308973	test-mlogloss:0.503874
[1658]	train-mlogloss:0.308862	test-mlogloss:0.503884
[1659]	train-mlogloss:0.308793	test-mlogloss:0.503911
[1660]	train-mlogloss:0.308701	test-mlogloss:0.503945
[1661]	train-mlogloss:0.308618	test-mlogloss:0.503959
[1662]	train-mlogloss:0.308541	test-mlogloss:0.503964
[1663]	train-mlogloss:0.308461	test-mlogloss:0.503942
[1664]	train-mlogloss:0.308352	test-mlogloss:0.503936
[1665]	train-mlogloss:0.308242	test-mlogloss:0.503947
[1666]	train-mlogloss:0.308165	test-mlogloss:0.50395
[1667]	train-mlogloss:0.308106	test-mlogloss:0.503932
[1668]	train-mlogloss:0.30801	test-mlogloss:0.503948
[1669]	train-mlogloss:0.307918	test-mlogloss:0.503942
[1670]	train-mlogloss:0.307826

In [81]:
print(cv_scores)
print('Mean loss: ' + str(np.mean(cv_scores)))

[0.51673298967790526, 0.50997789783597625, 0.51910585817278243, 0.50542327814429155, 0.50388515808686873]
Mean loss: 0.511025036384


In [ ]:
preds, model = runXGB(train_X, train_y, test_X, num_rounds=1600)
out_df = pd.DataFrame(preds)
out_df.columns = ["high", "medium", "low"]
out_df["listing_id"] = test_df.listing_id.values
out_df.to_csv("data/sub.csv", index=False)